<a href="https://colab.research.google.com/github/chi-hun/dacon/blob/main/%EC%84%9C%EC%9A%B8_%EB%9E%9C%EB%93%9C%EB%A7%88%ED%81%AC_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EB%B6%84%EB%A5%98(pytorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

서울 랜드마크 이미지 분류 https://dacon.io/competitions/official/235957/overview/description

# module

In [1]:
import pandas as pd
import numpy as np
import torch
import torchvision
import PIL
import glob
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
torch.manual_seed(42)

In [23]:
device = torch.device('cuda')

# data load

In [5]:
train_csv = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/데이콘/서울 렌드마크/train.csv')

In [65]:
train_csv

,file_name,label
0,001.PNG,9
1,002.PNG,4
2,003.PNG,1
3,004.PNG,1
4,005.PNG,6
...,...,...
718,719.PNG,9
719,720.PNG,6
720,721.PNG,2
721,722.PNG,8


In [6]:
path1 = '/content/drive/MyDrive/Colab Notebooks/데이콘/서울 렌드마크/train'
path_train = sorted(glob.glob(path1 + '/*/*'), key=lambda x: int(x.split('/')[-1].split('.')[0]))
path2 = '/content/drive/MyDrive/Colab Notebooks/데이콘/서울 렌드마크/test'
path_test = sorted(glob.glob(path2 + '/*'), key=lambda x: int(x.split('/')[-1].split('.')[0]))

In [50]:
class custom_dataset(torch.utils.data.Dataset):
    def __init__(self,data_path,label,train_mode=True, transform=None):
        self.data_path = data_path
        self.label = label
        self.transform = transform
        self.train_mode = train_mode

    def __getitem__(self, index):
        data_path_get = self.data_path[index]
        image = np.array(PIL.Image.open(data_path_get))
        
        if self.transform is not None:
            image = self.transform(image)
        
        if self.train_mode :
            assert data_path_get.split('/')[-1] == self.label['file_name'].values[index]
            label_get = self.label['label'].values[index]
            return image, label_get
        
        else:
            return image

    def __len__(self):
        return len(self.data_path)

In [8]:
transformer_train = torchvision.transforms.Compose([torchvision.transforms.ToPILImage(),
                                                    torchvision.transforms.Resize((128,128)),
                                                    torchvision.transforms.ToTensor(),
                                                    torchvision.transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
                                                    torchvision.transforms.RandomRotation(30),
                                                    torchvision.transforms.RandomHorizontalFlip()])
transformer_test = torchvision.transforms.Compose([torchvision.transforms.ToPILImage(),
                                                  torchvision.transforms.Resize((128,128)),
                                                  torchvision.transforms.ToTensor(),
                                                  torchvision.transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5))])

In [9]:
len_train = round(len(path_train) * 0.75)

path_ttrain = path_train[:len_train]
label_ttrain = train_csv.iloc[:len_train,:]

path_val = path_train[len_train:]
label_val = train_csv.iloc[len_train:,:]

In [10]:
dataest_train = custom_dataset(path_ttrain,label_ttrain,train_mode=True,transform=transformer_train)
dataloder_train = torch.utils.data.DataLoader(dataest_train, batch_size=32, shuffle=True)

dataset_val = custom_dataset(path_val,label_val, train_mode=True, transform=transformer_test)
dataloder_val = torch.utils.data.DataLoader(dataset_val,batch_size=32, shuffle=False)

In [51]:
dataest_test=custom_dataset(path_test,None,train_mode=False,transform=transformer_test)
dataloder_test=torch.utils.data.DataLoader(dataest_test,batch_size=32,shuffle=False)

# modeling

In [61]:
resnet = torchvision.models.resnet50(pretrained=True).to(device)
for param in resnet.parameters():
    param.requies_grad = False

in_features = resnet.fc.in_features

classifier = torch.nn.Sequential(torch.nn.Linear(in_features,1024),
                                 torch.nn.ReLU(),
                                 torch.nn.Dropout(0.5),
                                 torch.nn.Linear(1024,10))
resnet.fc = classifier

losscol = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=resnet.parameters(), lr=2e-2)
resnet.to(device)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [25]:
from torchsummary import summary
summary(resnet,(3,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           9,408
       BatchNorm2d-2           [-1, 64, 64, 64]             128
              ReLU-3           [-1, 64, 64, 64]               0
         MaxPool2d-4           [-1, 64, 32, 32]               0
            Conv2d-5           [-1, 64, 32, 32]           4,096
       BatchNorm2d-6           [-1, 64, 32, 32]             128
              ReLU-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
             ReLU-10           [-1, 64, 32, 32]               0
           Conv2d-11          [-1, 256, 32, 32]          16,384
      BatchNorm2d-12          [-1, 256, 32, 32]             512
           Conv2d-13          [-1, 256, 32, 32]          16,384
      BatchNorm2d-14          [-1, 256,

In [62]:
def train(model, optimizer, losscol, epochs, train_loder, val_loder, scheduler, device):
    model.to(device)
    n = len(train_loder)
    best_loss = 1e5
    for epoch in range(epochs):
        model.train()
        losses_train = 0
        accuracy_train = 0
        for x_train, y_train in tqdm(train_loder,colour='green'):
            x_train, y_train = x_train.to(device), y_train.to(device)

            optimizer.zero_grad()
            pre = model(x_train)
            loss = losscol(pre, y_train)
            loss.backward()
            optimizer.step()
            losses_train += loss.item()
            
            accuracy_train += (torch.argmax(pre, dim=1) == y_train).sum()

        if scheduler is not None:
            scheduler.step()
        model.eval()
        losses_val = 0
        accuracy_val = 0
        with torch.no_grad():
            for x_val, y_val in val_loder:
                x_val, y_val = x_val.to(device), y_val.to(device)

                pre = model(x_val)
                loss = losscol(pre, y_val)
                losses_val += loss.item()
                accuracy_val += (torch.argmax(pre, dim=1) == y_val).sum()
        print(f'{epoch+1:3d}/{epochs} train_loss : {losses_train/n:.4f}, train_accuracy : {accuracy_train/len(train_loder.dataset):.4f}, val_loss : {losses_val/n:.4f}, val_accuracy : {accuracy_val/len(val_loder.dataset):.4f}')

        if best_loss > losses_val/n :
            best_loss = losses_val/n
            torch.save(model.state_dict(),'/content/model_state.pth')
            print('save model')            

In [64]:
train(resnet,optimizer=optimizer, losscol=losscol, epochs=100, train_loder=dataloder_train, val_loder=dataloder_val, scheduler=None, device=device)

100%|██████████| 17/17 [00:15<00:00,  1.13it/s]


  1/100 train_loss : 1.2456, train_accuracy : 0.8081, val_loss : 0.3046, val_accuracy : 0.8453
save model


100%|██████████| 17/17 [00:15<00:00,  1.13it/s]


  2/100 train_loss : 0.5129, train_accuracy : 0.9539, val_loss : 0.1503, val_accuracy : 0.9337
save model


100%|██████████| 17/17 [00:15<00:00,  1.09it/s]


...

 28/100 train_loss : 0.0084, train_accuracy : 0.9982, val_loss : 0.0087, val_accuracy : 0.9945
save model


100%|██████████| 17/17 [00:14<00:00,  1.14it/s]


 29/100 train_loss : 0.0076, train_accuracy : 1.0000, val_loss : 0.0074, val_accuracy : 0.9945
save model


 71%|███████   | 12/17 [00:10<00:04,  1.15it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'PngImageFile'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  # Remove the CWD from sys.path while we load stuff.
 71%|███████   | 12/17 [00:11<00:04,  1.08it/s]


ValueError: ignored

In [56]:
def predict(model, data_loder, device):
    model.eval()
    pred = []
    with torch.no_grad():
        for x_test in data_loder:
            x_test = x_test.to(device)
            pre = model(x_test)
            pred_logit=torch.argmax(pre, dim=1,keepdim=True).squeeze(1)
            pred.extend(pred_logit.tolist())
    return pred

In [57]:
checkpoint=torch.load('/content/model_state.pth')
Predictor=resnet.to(device)
Predictor.load_state_dict(checkpoint)

preds=predict(Predictor,dataloder_test,device)

In [58]:
sub=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/데이콘/서울 렌드마크/sample_submission.csv')
sub['label']=preds
sub.to_csv('sub.csv',index=False)